In [ ]:
import numpy as np
import cv2
import sys
import matplotlib.pyplot as plt

### 히스토그램 역행

In [ ]:
src = cv2.imread("./fig/cropland.png")

src_ycrcb = cv2.cvtColor(src, cv2.COLOR_BGR2YCrCb)

x, y, w, h = cv2.selectROI(src)

crop = src_ycrcb[y : y + h, x : x + w]

size = [256, 256]
ranges = [0, 256, 0, 256]

hist = cv2.calcHist([crop], [1, 2], None, size, ranges)
backproj = cv2.calcBackProject([src_ycrcb], [1, 2], hist, ranges, 1)

dst = cv2.copyTo(src, backproj)

cv2.imshow("src", src)
cv2.imshow("backproj", backproj)
cv2.imshow("dst", dst)

cv2.waitKey()
cv2.destroyAllWindows()

### 필터 - FILTERS

In [ ]:
src = cv2.imread("./fig/blue_eyes.png", 0)

# kernel = np.ones((3, 3), np.float32) / 9
# dst = cv2.filter2D(src, -1, kernel) # 컨볼루션

dst = cv2.blur(src, (7, 7))
dst_g = cv2.GaussianBlur(src, (0, 0), 1)
dst_sharp = cv2.addWeighted(src, 2, dst_g, -1, 0) # 0은 beta value
dst_bila = cv2.bilateralFilter(src, -1, 5, 1) # 5은 sigmacolor, 1은 sigmaspace

cv2.imshow("src", src)
cv2.imshow("dst", dst)
cv2.imshow("dst_g", dst_g)
cv2.imshow("dst_sharp", dst_sharp)
cv2.imshow("dst_bilateral", dst_bila)

cv2.waitKey()
cv2.destroyAllWindows()

### 기하학적 변환

In [ ]:
src = cv2.imread("./fig/puppy.bmp", 1)

affine = np.array([[1, 0, 20],
                  [0, 1, 100]], np.float32)
dst = cv2.warpAffine(src, affine, (0, 0)) # 0, 0은 확대 비율

cv2.imshow("src", src)
cv2.imshow("dst", dst)

cv2.waitKey()
cv2.destroyAllWindows()

In [ ]:
src = cv2.imread("./fig/rose.jpg", 1)

dst_n = cv2.resize(src, (1920, 1080), interpolation = cv2.INTER_NEAREST)
dst_c = cv2.resize(src, (1920, 1080), interpolation = cv2.INTER_CUBIC)

cv2.imshow("src", src)
cv2.imshow("dst_n", dst_n)
cv2.imshow("dst_c", dst_c)

cv2.waitKey()
cv2.destroyAllWindows()

### 회전변환

In [ ]:
src = cv2.imread("./fig/puppy.bmp", 1)

h, w = src.shape[:2]

cp = (w // 2, h // 2)
affine = cv2.getRotationMatrix2D(cp, 20, 1)
dst = cv2.warpAffine(src, affine, (0, 0))

cv2.imshow("src", src)
cv2.imshow("dst_rot", dst)

cv2.waitKey()
cv2.destroyAllWindows()

# DNN

In [ ]:
import cv2
import numpy as np
import sys

In [ ]:
# googleNET 영상 인식
# imagesize = 224 x 224, color = BGR, mean(brightness) : (104, 117, 123)
filename = "./DNN/googlenet/fig/apple1.png"
img = cv2.imread(filename)

if img is None:
    print("image read failed")
    sys.exit()
    
# 사용할 네트워크(모델) - caffe 불러들이기
model = "./DNN/googlenet/bvlc_googlenet.caffemodel"
config = "./DNN/googlenet/deploy.prototxt"

net = cv2.dnn.readNet(model, config)

if net.empty():
    print("Net read failed")
    sys.exit()
    
# classnames 불러오기
classnames = []
with open("./DNN/googlenet/classification_classes_ILSVRC2012.txt") as f:
    classnames = f.read().rstrip("\n").split("\n")
# print(len(classnames)) -> 1000

# image 불러올 때 전처리 : blob image 만들기
# imagesize = 224 x 224, color = BGR, mean(brightness) : (104, 117, 123)
# cv2.dnn.blobFromImage(
# image[, scalefactor[, size[, mean[, swapRB[, crop[, ddepth]]]]]]) -> retval

blob = cv2.dnn.blobFromImage(img, 1, (224, 224), (104, 117, 123),
                            swapRB = False)

# net instance에 input 넣기
net.setInput(blob)

# network 진행
probability = net.forward()
# print(type(probability), probability.shape, probability[:, 10])

out = probability.flatten()
classid = np.argmax(out)
confidence = out[classid]
# print(confidence) -> 0.9361577

name = classnames[classid]
# print(name) -> pomegranate

text = "({}, {}%)".format(name, round(confidence * 100, 2))
cv2.putText(img, text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255),
           1, cv2.LINE_AA)

cv2.imshow("img", img)

cv2.waitKey()
cv2.destroyAllWindows()

In [ ]:
import os

file_list = os.listdir("./DNN/googlenet/fig/")

filenames = []
for i in file_list:
    file_name = "./DNN/googlenet/fig/" + i
    filenames.append(file_name)

idx = 0
while 1:
    img = cv2.imread(filenames[idx])

    if img is None:
        print("image read failed")
        sys.exit()

    model = "./DNN/googlenet/bvlc_googlenet.caffemodel"
    config = "./DNN/googlenet/deploy.prototxt"

    net = cv2.dnn.readNet(model, config)

    if net.empty():
        print("Net read failed")
        sys.exit()

    classnames = []
    with open("./DNN/googlenet/classification_classes_ILSVRC2012.txt") as f:
        classnames = f.read().rstrip("\n").split("\n")
    blob = cv2.dnn.blobFromImage(img, 1, (224, 224), (104, 117, 123),
                                swapRB = False)

    net.setInput(blob)

    probability = net.forward()

    out = probability.flatten()
    classid = np.argmax(out)
    confidence = out[classid]

    name = classnames[classid]

    text = "({}, {}%)".format(name, round(confidence * 100, 2))
    cv2.putText(img, text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255),
               1, cv2.LINE_AA)

    cv2.imshow("img", img)
    
    if cv2.waitKey(3000) == 27:
        break
        
    idx += 1
    
    if idx >= len(filenames):
        idx = 0
        
cv2.destroyAllWindows()

# Face recognition

In [ ]:
import cv2
import numpy as np
import sys
import os

### caffe face detection model

In [ ]:
img = cv2.imread("./DNN/googlenet/fig/mb.jpg")

model = "./DNN/opencv_face_detector/res10_300x300_ssd_iter_140000_fp16.caffemodel"
config = "./DNN/opencv_face_detector/deploy.prototxt"

# readNet(model[, config[, framework]]) -> retval
facenet = cv2.dnn.readNet(model, config)

if facenet.empty():
    print("net open failed")
    sys.exit()
    
blob = cv2.dnn.blobFromImage(img, 1, # 1은 scalefactor
                             (300, 300), # image size 
                             (104, 177, 123), # 
                             swapRB = False) # RGB로 할 것인가?

facenet.setInput(blob)
out = facenet.forward()
# print(out.shape) -> (1, 1, 200, 7), idx0, idx1은 안씀

detect = out[0, 0, :, :]
h, w = img.shape[:2]

for i in range(detect.shape[0]):
    confidence = detect[i, 2]
    
    if confidence > 0.5:
        x1 = int(detect[i, 3] * w)
        y1 = int(detect[i, 4] * h)
        x2 = int(detect[i, 5] * w)
        y2 = int(detect[i, 6] * h)
        
        cv2.rectangle(img, (x1, y1), (x2, y2), (0, 0, 255))
        text = "face : {}%".format(round(confidence * 100, 2))
        
        cv2.putText(img, text, (x1, y1 - 3), cv2.FONT_HERSHEY_SIMPLEX,
                   0.8, (0, 0, 255), 1, cv2.LINE_AA)

if img is None:
    print("image read failed")
    sys.exit()
    
cv2.imshow("img", img)

while 1:
    if cv2.waitKey() == 27:
        break

cv2.destroyAllWindows()

### pb face detection model

In [ ]:
img = cv2.imread("./DNN/googlenet/fig/mb.jpg")

model = "./DNN/opencv_face_detector/opencv_face_detector_uint8.pb"
config = "./DNN/opencv_face_detector/opencv_face_detector.pbtxt"

# readNet(model[, config[, framework]]) -> retval
facenet = cv2.dnn.readNet(model, config)

if facenet.empty():
    print("net open failed")
    sys.exit()
    
blob = cv2.dnn.blobFromImage(img, 1, # 1은 scalefactor
                             (300, 300), # image size 
                             (104, 177, 123), # 
                             swapRB = False) # RGB로 할 것인가?

facenet.setInput(blob)
out = facenet.forward()
# print(out.shape) -> (1, 1, 200, 7), idx0, idx1은 안씀

detect = out[0, 0, :, :]
h, w = img.shape[:2]

for i in range(detect.shape[0]):
    confidence = detect[i, 2]
    
    if confidence > 0.5:
        x1 = int(detect[i, 3] * w)
        y1 = int(detect[i, 4] * h)
        x2 = int(detect[i, 5] * w)
        y2 = int(detect[i, 6] * h)
        
        cv2.rectangle(img, (x1, y1), (x2, y2), (0, 0, 255))
        text = "face : {}%".format(round(confidence * 100, 2))
        
        cv2.putText(img, text, (x1, y1 - 3), cv2.FONT_HERSHEY_SIMPLEX,
                   0.8, (0, 0, 255), 1, cv2.LINE_AA)

if img is None:
    print("image read failed")
    sys.exit()
    
cv2.imshow("img", img)

# while 1:
#     if cv2.waitKey() == 27:
#         break

cv2.waitKey()
cv2.destroyAllWindows()

### webcam face detection

In [2]:
import cv2
import numpy as np
import sys
import os

cap = cv2.VideoCapture(0)
cap.isOpened()

while 1:
    returnvalue, frame = cap.read()
    if not returnvalue:
        print("video read failed")
        break

    model = "./DNN/opencv_face_detector/opencv_face_detector_uint8.pb"
    config = "./DNN/opencv_face_detector/opencv_face_detector.pbtxt"

    # readNet(model[, config[, framework]]) -> retval
    facenet = cv2.dnn.readNet(model, config)

    if facenet.empty():
        print("net open failed")
        sys.exit()

    blob = cv2.dnn.blobFromImage(frame, 1, # 1은 scalefactor
                                 (300, 300), # image size 
                                 (104, 177, 123), # 
                                 swapRB = False) # RGB로 할 것인가?

    facenet.setInput(blob)
    out = facenet.forward()
    # print(out.shape) -> (1, 1, 200, 7), idx0, idx1은 안씀

    detect = out[0, 0, :, :]
    h, w = frame.shape[:2]

    for i in range(detect.shape[0]):
        confidence = detect[i, 2]

        if confidence > 0.5:
            x1 = int(detect[i, 3] * w)
            y1 = int(detect[i, 4] * h)
            x2 = int(detect[i, 5] * w)
            y2 = int(detect[i, 6] * h)

            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255))
            text = "face : {}%".format(round(confidence * 100, 2))

            cv2.putText(frame, text, (x1, y1 - 3), cv2.FONT_HERSHEY_SIMPLEX,
                       0.8, (0, 0, 255), 1, cv2.LINE_AA)

    if frame is None:
        print("image read failed")
        sys.exit()

    cv2.imshow("img", frame)

    if cv2.waitKey(30) == 27:
        break
    
cap.release()
cv2.destroyAllWindows()

### object detection YOLO3

In [5]:
import glob

In [17]:
model = "./DNN/yolo_v3/yolov3.weights"
config = "./DNN/yolo_v3/yolov3.cfg"
class_labels = "./DNN/yolo_v3/coco.names"

img_files = glob.glob("./DNN/yolo_v3/fig/*.jpg")
# print(img_files) ->
# ['./DNN/yolo_v3/fig\\dog.jpg', './DNN/yolo_v3/fig\\kite.jpg', 
# './DNN/yolo_v3/fig\\person.jpg', './DNN/yolo_v3/fig\\sheep.jpg']

net = cv2.dnn.readNet(model, config)

if net.empty():
    print("net read failed")
    sys.exit()
    
classes = []
with open(class_labels, "rt") as f:
    classes = f.read().rstrip("\n").split("\n")

colors = np.random.uniform(0, 255, size = (80, 3))
# print(colors)

layer_names = net.getLayerNames()
# print(layer_names)

output_layers = [layer_names[i-1] for i in net.getUnconnectedOutLayers()]
# print(output_layers)

for i in img_files:
    img = cv2.imread(i)
    
    if img is None:
        continue
        
    blob = cv2.dnn.blobFromImage(img, 1/255., (320, 320), swapRB = True)
    net.setInput(blob)
    outs = net.forward(output_layers)
    
    h, w = img.shape[:2]
    
    class_ids = []
    confidences = []
    boxes = []
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            
            if confidence > 0.5:
                cx = int(detection[0] * w)
                cy = int(detection[1] * h)
                bw = int(detection[2] * w)
                bh = int(detection[3] * h)
                
                sx = int(cx - bw // 2)
                sy = int(cy - bh // 2)
                
                boxes.append([sx, sy, bw, bh])
                confidences.append(float(confidence))
                class_ids.append(int(class_id))
                
    indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    
    for i in indices:
#         i = i[0]
        sx, sy, bw, bh = boxes[i]
        label = "{}:{}".format(classes[class_ids[i]], round(confidences[i], 2))
        
        color = colors[class_ids[i]]
        
        cv2.rectangle(img, (sx, sy, bw, bh), color, 2)
        cv2.putText(img, label, (sx, sy - 3), cv2.FONT_HERSHEY_SIMPLEX,
                   0.8, (0, 0, 255), 1, cv2.LINE_AA)
    t, _ = net.getPerfProfile()
    label = "inference time : {}".format(t * 1000.0 / cv2.getTickFrequency())
    cv2.putText(img, label, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1,
               (255, 0, 0), 1, cv2.LINE_AA)
    
    cv2.imshow("omg", img)
    cv2.waitKey()
    
cv2.destroyAllWindows()